# 解谜游戏： Simple Loop


WARNING: NOT FINISHED! 
- 用一条连续线路串联起每个白色格子。必须形成一个环路。
- 线只能从上下左右格子走，不能对角线走；
- 实际上是找一个汉密尔顿回路，需要修改代码
---------

In [ ]:
from ortools.sat.python import cp_model as cp

def simpleloop_solver(grid, X, Y):
    try:
        len(grid) == X * Y
    except Exception:
        raise(f"检查输入棋盘的长度。length of grid != {X} * {Y}")
    
    model = cp.CpModel()
    x = {}
    
    
    for i in range(Y):
        for j in range(X):
            if grid[i * X + j] == "1":
                x[i, j, 1, 0] = 0
                x[i, j, -1, 0] = 0
                x[i, j, 0, 1] = 0
                x[i, j, 0, -1] = 0
                model.Add(x[i, j , 1, 0] + x[i, j, -1, 0] + x[i, j, 0, -1] + x[i, j, 0, 1] == 0)
                
            else:
                if i + 1 < Y:
                    x[i, j, 1, 0] = model.NewBoolVar(name = f"x[{i}, {j}, 1, 0]")
                else:
                    x[i, j, 1, 0] = 0
                
                if i - 1 >= 0:
                    x[i, j , -1, 0] = model.NewBoolVar(name = f"x[{i}, {j}, -1, 0]")
                else:
                    x[i, j , -1, 0] = 0
                    
                if j + 1 < X:
                    x[i, j , 0, 1] = model.NewBoolVar(name = f"x[{i}, {j}, 0, 1]")
                else:
                    x[i, j , 0, 1] = 0
                
                if j - 1 >= 0:
                    x[i, j , 0, -1] = model.NewBoolVar(name = f"x[{i}, {j}, 0, -1]")
                else:
                    x[i, j , 0, -1] = 0
                    
                model.Add(x[i, j , 1, 0] + x[i, j, -1, 0] + x[i, j, 0, -1] + x[i, j, 0, 1] == 2)
        
    
    for i in range(Y):
        for j in range(X):
            if j - 1 >= 0:
                model.Add(x[i, j, 0, -1] == x[i, j - 1, 0, 1]).OnlyEnforceIf(x[i, j , 0, -1])
            if j + 1 < X:
                model.Add(x[i, j, 0, 1] == x[i, j + 1, 0, -1]).OnlyEnforceIf(x[i, j , 0, 1])
            if i + 1 < Y:
                model.Add(x[i, j, 1, 0] == x[i + 1, j, -1, 0]).OnlyEnforceIf(x[i, j, 1, 0])
            if i - 1 >= 0:
                model.Add(x[i, j, -1, 0] == x[i - 1, j, 1, 0]).OnlyEnforceIf(x[i, j, -1, 0])

    solver = cp.CpSolver()
    status = solver.Solve(model)
    cn = 0
    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                # print(solver.Value(y[i, j]), end = " ")
                if solver.Value(x[i, j, 0, -1]) and solver.Value(x[i, j, 0, 1]):
                    print("-", end = "")
                elif solver.Value(x[i, j, 0, -1]) and solver.Value(x[i, j, -1, 0]):
                    print("┘", end = "")
                elif solver.Value(x[i, j, 0, -1]) and solver.Value(x[i, j, 1, 0]):
                    print("┐", end = "")
                elif solver.Value(x[i, j, -1, 0]) and solver.Value(x[i, j, 1, 0]):
                    print("|", end = "")
                elif solver.Value(x[i, j, -1, 0]) and solver.Value(x[i, j, 0, 1]):
                    print("└", end = "")
                elif solver.Value(x[i, j, 1, 0]) and solver.Value(x[i, j, 0, 1]):
                    print("┌", end = "")
                else:
                    # if grid[i * Y + j] == "1":
                    #     print("FAS!")
                    # else:
                    
                    print("a", end = "")
            print()
        print()
        
        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())

    else:
        print("Unable to find the OPTIMAL.")
    
    print(cn)
if __name__ == "__main__":
    
    grid = "0010001000010000000000000000000000000000000000010010010000000000000000000100101000000001000000000000000000000100001000000001000010000001100000000000001000000001010010000000000000000000100000010000000000000010000001000000000000000000000000100100000010000000000000000000000000000010000001000"

    simpleloop_solver(grid, 17, 17)